# Neptune as Graph Memory

In this notebook, we will be connecting using a Amazon Neptune Analytics instance as our memory graph storage for Mem0.

The Graph Memory storage persists memories in a graph or relationship form when performing `m.add` memory operations. It then uses vector distance algorithms to find related memories during a `m.search` operation. Relationships are returned in the result, and add context to the memories.

Reference: [Vector Similarity using Neptune Analytics](https://docs.aws.amazon.com/neptune-analytics/latest/userguide/vector-similarity.html)

## Prerequisites

### 1. Install Mem0 with Graph Memory support 

To use Mem0 with Graph Memory support, install it using pip:

```bash
pip install "mem0ai[graph]"
```

This command installs Mem0 along with the necessary dependencies for graph functionality.

### 2. Connect to Neptune

To connect to Amazon Neptune Analytics, you need to configure Neptune with your Amazon profile credentials. The best way to do this is to declare environment variables with IAM permission to your Neptune Analytics instance. The `graph-identifier` for the instance to persist memories needs to be defined in the Mem0 configuration under `"graph_store"`, with the `"neptune"` provider.  Note that the Neptune Analytics instance needs to have `vector-search-configuration` defined to meet the needs of the llm model's vector dimensions, see: https://docs.aws.amazon.com/neptune-analytics/latest/userguide/vector-index.html.

```python
embedding_dimensions = 1536
graph_identifier = "<MY-GRAPH>" # graph with 1536 dimensions for vector search
config = {
    "embedder": {
        "provider": "openai",
        "config": {
            "model": "text-embedding-3-large",
            "embedding_dims": embedding_dimensions
        },
    },
    "graph_store": {
        "provider": "neptune",
        "config": {
            "endpoint": f"neptune-graph://{graph_identifier}",
        },
    },
}
```

### 3. Configure OpenSearch

We're going to use OpenSearch as our vector store.  You can run [OpenSearch from docker image](https://docs.opensearch.org/docs/latest/install-and-configure/install-opensearch/docker/):

```bash
docker pull opensearchproject/opensearch:3
```

And verify that it's running with a `<custom-admin-password>`:

```bash
 docker run -d -p 9200:9200 -p 9600:9600 -e "discovery.type=single-node" -e "OPENSEARCH_INITIAL_ADMIN_PASSWORD=<custom-admin-password>" opensearchproject/opensearch:latest

 curl https://localhost:9200 -ku admin:<custom-admin-password>
```

We're going to connect [OpenSearch using the python client](https://github.com/opensearch-project/opensearch-py):

```bash
pip install "opensearch-py"
```

## Configuration

Do all the imports and configure OpenAI (enter your OpenAI API key):

In [1]:
from mem0 import Memory
from langchain_aws import NeptuneAnalyticsGraph
import os
import logging
import sys

logging.getLogger("mem0.memory.neptune_memory").setLevel(logging.DEBUG)
logging.getLogger("mem0.memory.neptune_base").setLevel(logging.DEBUG)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

logging.basicConfig(
    format="%(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    stream=sys.stdout,  # Explicitly set output to stdout
)

Setup the Mem0 configuration using:
- openai as the embedder
- Amazon Neptune Analytics instance as a graph store
- OpenSearch as the vector store

In [2]:
graph_identifier = os.environ.get("GRAPH_ID")
opensearch_username = os.environ.get("OS_USERNAME")
opensearch_password = os.environ.get("OS_PASSWORD")
config = {
    "embedder": {
        "provider": "openai",
        "config": {"model": "text-embedding-3-large", "embedding_dims": 1536},
    },
    "graph_store": {
        "provider": "neptune",
        "config": {
            "endpoint": f"neptune-graph://{graph_identifier}",
        },
    },
    "vector_store": {
        "provider": "opensearch",
        "config": {
            "collection_name": "vector_store",
            "host": "localhost",
            "port": 9200,
            "user": opensearch_username,
            "password": opensearch_password,
            "use_ssl": False,
            "verify_certs": False,
        },
    },
}

neptune_graph = NeptuneAnalyticsGraph(graph_identifier)

/Users/andrewc/git/acarbonetto/mem0/.venv/lib/python3.12/site-packages/botocore/auth.py:425: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime_now = datetime.datetime.utcnow()


## Graph Memory initializiation

Initialize Memgraph as a Graph Memory store:

In [3]:
m = Memory.from_config(config_dict=config)

app_id = "movies"
user_id = "alice"

m.delete_all(user_id=user_id)

DEBUG - delete_all query=
        MATCH (n  {user_id: $user_id})
        DETACH DELETE n
        


{'message': 'Memories deleted successfully!'}

## Store memories

Create memories and store one at a time:

In [4]:
messages = [
    {
        "role": "user",
        "content": "I'm planning to watch a movie tonight. Any recommendations?",
    },
]

# Store inferred memories (default behavior)
result = m.add(messages, user_id=user_id, metadata={"category": "movie_recommendations"})

all_results = m.get_all(user_id=user_id)
for n in all_results["results"]:
    print(f"node \"{n["memory"]}\": [hash: {n["hash"]}]")

for e in all_results["relations"]:
    print(f"edge \"{e["source"]}\" --{e["relationship"]}--> \"{e["target"]}\"")

DEBUG - Extracted entities: [{'source': 'alice', 'relationship': 'plans_to_watch', 'destination': 'movie'}, {'source': 'alice', 'relationship': 'plans_to_watch', 'destination': 'tonight'}]
DEBUG - _search_graph_db
  query=
            MATCH (n )
            WHERE n.user_id = $user_id
            WITH n, $n_embedding as n_embedding
            CALL neptune.algo.vectors.distanceByEmbedding(
                n_embedding,
                n,
                {metric:"CosineSimilarity"}
            ) YIELD distance
            WITH n, distance as similarity
            WHERE similarity >= $threshold
            CALL {
                WITH n
                MATCH (n)-[r]->(m) 
                RETURN n.name AS source, id(n) AS source_id, type(r) AS relationship, id(r) AS relation_id, m.name AS destination, id(m) AS destination_id
                UNION ALL
                WITH n
                MATCH (m)-[r]->(n) 
                RETURN m.name AS source, id(m) AS source_id, type(r) AS relationshi

In [7]:
messages = [
    {
        "role": "assistant",
        "content": "How about a thriller movies? They can be quite engaging.",
    },
]

# Store inferred memories (default behavior)
result = m.add(messages, user_id=user_id, metadata={"category": "movie_recommendations"})

all_results = m.get_all(user_id=user_id)
for n in all_results["results"]:
    print(f"node \"{n["memory"]}\": [hash: {n["hash"]}]")

for e in all_results["relations"]:
    print(f"edge \"{e["source"]}\" --{e["relationship"]}--> \"{e["target"]}\"")

DEBUG - Extracted entities: [{'source': 'thriller_movies', 'relationship': 'is_engaging', 'destination': 'thriller_movies'}]
DEBUG - _search_graph_db
  query=
            MATCH (n )
            WHERE n.user_id = $user_id
            WITH n, $n_embedding as n_embedding
            CALL neptune.algo.vectors.distanceByEmbedding(
                n_embedding,
                n,
                {metric:"CosineSimilarity"}
            ) YIELD distance
            WITH n, distance as similarity
            WHERE similarity >= $threshold
            CALL {
                WITH n
                MATCH (n)-[r]->(m) 
                RETURN n.name AS source, id(n) AS source_id, type(r) AS relationship, id(r) AS relation_id, m.name AS destination, id(m) AS destination_id
                UNION ALL
                WITH n
                MATCH (m)-[r]->(n) 
                RETURN m.name AS source, id(m) AS source_id, type(r) AS relationship, id(r) AS relation_id, n.name AS destination, id(n) AS destina

In [8]:
messages = [
    {
        "role": "user",
        "content": "I'm not a big fan of thriller movies but I love sci-fi movies.",
    },
]

# Store inferred memories (default behavior)
result = m.add(messages, user_id=user_id, metadata={"category": "movie_recommendations"})

all_results = m.get_all(user_id=user_id)
for n in all_results["results"]:
    print(f"node \"{n["memory"]}\": [hash: {n["hash"]}]")

for e in all_results["relations"]:
    print(f"edge \"{e["source"]}\" --{e["relationship"]}--> \"{e["target"]}\"")

DEBUG - Extracted entities: [{'source': 'alice', 'relationship': 'not_a_fan_of', 'destination': 'thriller_movies'}, {'source': 'alice', 'relationship': 'loves', 'destination': 'sci-fi_movies'}]
DEBUG - _search_graph_db
  query=
            MATCH (n )
            WHERE n.user_id = $user_id
            WITH n, $n_embedding as n_embedding
            CALL neptune.algo.vectors.distanceByEmbedding(
                n_embedding,
                n,
                {metric:"CosineSimilarity"}
            ) YIELD distance
            WITH n, distance as similarity
            WHERE similarity >= $threshold
            CALL {
                WITH n
                MATCH (n)-[r]->(m) 
                RETURN n.name AS source, id(n) AS source_id, type(r) AS relationship, id(r) AS relation_id, m.name AS destination, id(m) AS destination_id
                UNION ALL
                WITH n
                MATCH (m)-[r]->(n) 
                RETURN m.name AS source, id(m) AS source_id, type(r) AS relati

In [9]:
messages = [
    {
        "role": "assistant",
        "content": "Got it! I'll avoid thriller recommendations and suggest sci-fi movies in the future.",
    },
]

# Store inferred memories (default behavior)
result = m.add(messages, user_id=user_id, metadata={"category": "movie_recommendations"})

all_results = m.get_all(user_id=user_id)
for n in all_results["results"]:
    print(f"node \"{n["memory"]}\": [hash: {n["hash"]}]")

for e in all_results["relations"]:
    print(f"edge \"{e["source"]}\" --{e["relationship"]}--> \"{e["target"]}\"")

DEBUG - Extracted entities: [{'source': 'alice', 'relationship': 'avoids', 'destination': 'thriller_recommendations'}, {'source': 'alice', 'relationship': 'suggests', 'destination': 'sci-fi_movies'}]
DEBUG - _search_graph_db
  query=
            MATCH (n )
            WHERE n.user_id = $user_id
            WITH n, $n_embedding as n_embedding
            CALL neptune.algo.vectors.distanceByEmbedding(
                n_embedding,
                n,
                {metric:"CosineSimilarity"}
            ) YIELD distance
            WITH n, distance as similarity
            WHERE similarity >= $threshold
            CALL {
                WITH n
                MATCH (n)-[r]->(m) 
                RETURN n.name AS source, id(n) AS source_id, type(r) AS relationship, id(r) AS relation_id, m.name AS destination, id(m) AS destination_id
                UNION ALL
                WITH n
                MATCH (m)-[r]->(n) 
                RETURN m.name AS source, id(m) AS source_id, type(r) AS 

## Search memories

In [16]:
search_results = m.search("what does alice love?", user_id=user_id)
for result in search_results["results"]:
    print(f"\"{result["memory"]}\" [score: {result["score"]}]")
for relation in search_results["relations"]:
    print(f"{relation}")

DEBUG - _search_graph_db
  query=
            MATCH (n )
            WHERE n.user_id = $user_id
            WITH n, $n_embedding as n_embedding
            CALL neptune.algo.vectors.distanceByEmbedding(
                n_embedding,
                n,
                {metric:"CosineSimilarity"}
            ) YIELD distance
            WITH n, distance as similarity
            WHERE similarity >= $threshold
            CALL {
                WITH n
                MATCH (n)-[r]->(m) 
                RETURN n.name AS source, id(n) AS source_id, type(r) AS relationship, id(r) AS relation_id, m.name AS destination, id(m) AS destination_id
                UNION ALL
                WITH n
                MATCH (m)-[r]->(n) 
                RETURN m.name AS source, id(m) AS source_id, type(r) AS relationship, id(r) AS relation_id, n.name AS destination, id(n) AS destination_id
            }
            WITH distinct source, source_id, relationship, relation_id, destination, destination_id, si

In [18]:
m.delete_all("user_id")
m.reset()

DEBUG - delete_all query=
        MATCH (n  {user_id: $user_id})
        DETACH DELETE n
        
WARNING - Resetting all memories
WARNING - Resetting index vector_store...
